In [145]:
import hashlib
import re
import time
import json

In [146]:
def getZeros(string):
    result = re.search(r"^0+", string)
    if result:
        return result.group(0)
    else:
        return ""

In [147]:
class BlockHeader:
    version = 1
    merkleRoot = ""
    timestamp = 0
    bits = "ffff001f"
    nonce = 0
    blockHash = ""
    prevBlockHash = ""
    
    def __init__(self, prevBlockHash):
        self.prevBlockHash = prevBlockHash
        self.timestamp = int(time.time())
        self.blockHash = self.makeHash()

    def mine(self):
        difficultyValue = 4
        while len(getZeros(self.blockHash)) != difficultyValue:
            self.nonce += 1
            self.blockHash = self.makeHash()
        

    def makeHash(self):
        preEncodeString = f"{self.version}{self.merkleRoot}{self.timestamp}{self.nonce}{self.prevBlockHash}"
        hashRoundOne = hashlib.sha256(preEncodeString.encode('utf-8')).hexdigest()
        hashRoundTwo = hashlib.sha256(hashRoundOne.encode('utf-8')).hexdigest()
        return str(hashRoundTwo)


In [148]:
class Block:
    blockHeader = ""
    blockSize = 1
    txCount = 0
    txs = ""
    height = 0
    
    def __init__(self, height, prevBlockHash):
        self.blockHeader = BlockHeader(prevBlockHash)
        self.height = height

    def mine(self):
        self.blockHeader.mine();
        self.txs = f"Andrii sent {self.height} coins to Alice"
        self.txCount += 1
        self.blockHeader.merkleRoot = str(hashlib.sha256(self.txs.encode('utf-8')).hexdigest())
        return self
    
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    def getBlockData(self):
        print(self.toJSON())
        return self

In [149]:
class Blockchain:
    chain = []

    def genesisBlock(self):
        zeroHash = "".rjust(64, "0")
        initialBlock = Block(0, zeroHash).mine().getBlockData()
        self.chain.append(initialBlock)
        return self

    def addBlock(self):
        previousBlock = self.chain[len(self.chain) - 1];
        newBlock = Block(previousBlock.height + 1, previousBlock.blockHeader.blockHash).mine().getBlockData();

        self.chain.append(newBlock);
        return self

In [150]:
blockchainInstance = Blockchain()
genesisBlock = blockchainInstance.genesisBlock()
genesisBlock.addBlock()
genesisBlock.addBlock()

{
    "blockHeader": {
        "blockHash": "0000af4793fa81b4dcc0ef907a3c539047fda1547246359d6504fcb5d26a739f",
        "merkleRoot": "440c15781fe1fd99a6b57d72d34ffb21d10a629e5e9dd69d1fe34bcd9f21affd",
        "nonce": 202679,
        "prevBlockHash": "0000000000000000000000000000000000000000000000000000000000000000",
        "timestamp": 1670184969
    },
    "height": 0,
    "txCount": 1,
    "txs": "Andrii sent 0 coins to Alice"
}
{
    "blockHeader": {
        "blockHash": "0000441861f3ee8b2c5c05d92e0d811d82afd4f6990161e28539044601f8319a",
        "merkleRoot": "550394aef6271f3cfb36e5b25f0bbff2fd2f8369522b9853518573c9a910e189",
        "nonce": 50545,
        "prevBlockHash": "0000af4793fa81b4dcc0ef907a3c539047fda1547246359d6504fcb5d26a739f",
        "timestamp": 1670184971
    },
    "height": 1,
    "txCount": 1,
    "txs": "Andrii sent 1 coins to Alice"
}
{
    "blockHeader": {
        "blockHash": "00004f87b76f8a99ec5af95c4d6f719686d0cd871a44bbe29114459da41af588",
        "merk